In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 4.1 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=8999a4c35f94567b6051210f1d9bf668ed9327e20f3e5d5d82e2069f16b78629
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [2]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [1]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv   a0606.csv	     heatmap530.ipynb
 a0530.csv   a0607.csv	     heatmap531.ipynb
 a0531.csv   a0608.csv	     Heat_Map_Online_language.ipynb
 a0601.csv   a0609.csv	     __MACOSX
 a0602.csv   csv	     non_violant1.csv
 a0603.csv   csv.zip	     non_violant2.csv
 a0604.csv   dataframe_csv   violant.csv
 a0605.csv   Github	    '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_610 = 'csv/US06_10.csv'

In [8]:
ct = pd.read_csv(ct_path_610)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,143272560685_10157509661910686,Facebook,2020-06-10 20:07:25,2021-04-21 11:36:51,live_video_scheduled,"Colin Kaepernick, Jamie Foxx & Al Sharpton Are...",papercitymag.com,George Floyd always wanted to change the world...,The House Judiciary Committee today held a hea...,1.0,https://www.facebook.com/thenicolesandlershow/...,https://www.facebook.com/143272560685/posts/10...,3517,-0.857143,2.0,0,1,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,297062,The Nicole Sandler Show,thenicolesandlershow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Nicole Sandler Show is LIVE weekdays 5-6pm...,False,2009-09-21 19:00:53,en,0,297062|10157509661910686
1,28940123736_10157631983143737,Facebook,2020-06-10 20:07:21,2021-07-11 23:25:40,link,Your Guide to Black-Owned Eateries Around the ...,7x7.com,"*UPDATE June 10, 2020: Due to the outrage caus...","Cupcakes for the cause, anyone?",1.0,https://www.7x7.com/black-owned-restaurants-ba...,https://www.facebook.com/28940123736/posts/101...,75042,2.157895,1.0,28,6,3,4,0,0,0,0,0,0,8,2,2,3,2,1,0,0,0,1,53679,7x7,7x7,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 2001, 7x7 is an online magazine cel...",True,2008-05-26 21:28:39,en,0,53679|10157631983143737
2,47899317691_10158344106037692,Facebook,2020-06-10 20:07:09,2021-08-31 04:19:06,link,Southfield invites participants in demonstrati...,theoaklandpress.com,The city of Southfield will sponsor a demonstr...,Attendees are requested to wear face masks and...,1.0,https://bit.ly/3hhkAZq,https://www.facebook.com/47899317691/posts/101...,36868,-4.428571,1.0,1,0,2,3,1,0,0,0,0,0,4,2,5,3,2,5,5,5,0,2,12578,The Oakland Press,TheOaklandPress,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Your source for news and information about Oak...,True,2009-01-29 17:36:35,en,0,12578|10158344106037692
3,1937062589693148_3117814444951284,Facebook,2020-06-10 20:07:04,2021-07-09 03:38:57,link,George Floyd's 6-year-old daughter was just of...,businessinsider.com,"George Floyd's daughter, Gianna, has been offe...","George Floyd's daughter, Gianna, has been offe...",1.0,https://trib.al/gTEcK5d,https://www.facebook.com/1937062589693148/post...,102066,-11.625000,1.0,4,1,0,3,0,0,0,0,0,0,2,2,2,77,2,4,2,2,0,21,4729586,Insider News,InsiderNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,What you want to know about news. A section of...,True,2018-09-07 16:16:50,en,0,4729586|3117814444951284
4,215148520550_10163930481965551,Facebook,2020-06-10 20:07:03,2021-09-01 22:03:05,link,Black Scholar Shelby Steele Says Biden's Georg...,westernjournal.com,'What is Mr. Biden doing?',"'Does he know anything at all, really, about t...",1.0,https://www.westernjournal.com/top-black-schol...,https://www.facebook.com/215148520550/posts/10...,258826,1.833333,1.0,104,38,16,1,2,6,6,3,0,0,41,10,10,10,3,5,3,14,0,8,856464,Patriot Update by WJ,patriotupdate,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Patriot Update by WJ is a conservative email n...,False,2009-07-23 13:55:23,en,0,856464|10163930481965551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11742,332113686997_1015

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{},{}
2,{},{'Southfield': 1}
3,{'Texas': 1},{'Texas': 1}
4,{},{}


In [24]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,143272560685_10157509661910686,Facebook,2020-06-10 20:07:25,2021-04-21 11:36:51,live_video_scheduled,"Colin Kaepernick, Jamie Foxx & Al Sharpton Are...",papercitymag.com,George Floyd always wanted to change the world...,The House Judiciary Committee today held a hea...,1.0,https://www.facebook.com/thenicolesandlershow/...,https://www.facebook.com/143272560685/posts/10...,3517,-0.857143,2.0,0,1,0,0,0,0,0,0,0,0,2,2,2,1,0,0,0,0,0,0,297062,The Nicole Sandler Show,thenicolesandlershow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Nicole Sandler Show is LIVE weekdays 5-6pm...,False,2009-09-21 19:00:53,en,0,297062|10157509661910686,{},{},{}
1,28940123736_10157631983143737,Facebook,2020-06-10 20:07:21,2021-07-11 23:25:40,link,Your Guide to Black-Owned Eateries Around the ...,7x7.com,"*UPDATE June 10, 2020: Due to the outrage caus...","Cupcakes for the cause, anyone?",1.0,https://www.7x7.com/black-owned-restaurants-ba...,https://www.facebook.com/28940123736/posts/101...,75042,2.157895,1.0,28,6,3,4,0,0,0,0,0,0,8,2,2,3,2,1,0,0,0,1,53679,7x7,7x7,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Founded in 2001, 7x7 is an online magazine cel...",True,2008-05-26 21:28:39,en,0,53679|10157631983143737,{},{},{}
2,47899317691_10158344106037692,Facebook,2020-06-10 20:07:09,2021-08-31 04:19:06,link,Southfield invites participants in demonstrati...,theoaklandpress.com,The city of Southfield will sponsor a demonstr...,Attendees are requested to wear face masks and...,1.0,https://bit.ly/3hhkAZq,https://www.facebook.com/47899317691/posts/101...,36868,-4.428571,1.0,1,0,2,3,1,0,0,0,0,0,4,2,5,3,2,5,5,5,0,2,12578,The Oakland Press,TheOaklandPress,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Your source for news and information about Oak...,True,2009-01-29 17:36:35,en,0,12578|10158344106037692,{},{'Southfield': 1},{'Southfield': 1}
3,1937062589693148_3117814444951284,Facebook,2020-06-10 20:07:04,2021-07-09 03:38:57,link,George Floyd's 6-year-old daughter was just of...,businessinsider.com,"George Floyd's daughter, Gianna, has been offe...","George Floyd's daughter, Gianna, has been offe...",1.0,https://trib.al/gTEcK5d,https://www.facebook.com/1937062589693148/post...,102066,-11.625000,1.0,4,1,0,3,0,0,0,0,0,0,2,2,2,77,2,4,2,2,0,21,4729586,Insider News,InsiderNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,What you want to know about news. A section of...,True,2018-09-07 16:16:50,en,0,4729586|3117814444951284,{'Texas': 1},{'Texas': 1},{}
4,215148520550_10163930481965551,Facebook,2020-06-10 20:07:03,2021-09-01 22:03:05,link,Black Scholar Shelby Steele Says Biden's Georg...,westernjournal.com,'What is Mr. Biden doing?',"'Does he know anything at all, really, about t...",1.0,https://www.westernjournal.com/top-black-schol...,https://www.facebook.com/215148520550/posts/10...,258826,1.833333,1.0,104,38,16,1,2,6,6,3,0,0,41,10,10,10,3,5,3,14,0,8,856464,Patriot Update by WJ,patriotupdate,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Patriot Update by WJ is a conservative email n...,False,2009-07-23 13:55:23,en,0,856464|10163930481965551,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [25]:
ct.to_csv('a0610.csv')

In [26]:
ct=pd.read_csv('a0610.csv',lineterminator='\n')

In [27]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [28]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [29]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
2,{},{'Southfield': 1},{'Southfield': 1}
3,{'Texas': 1},{'Texas': 1},{}
5,{'Texas': 1},{'Texas': 1},{}
7,{'Westchester': 1},{},{'Westchester': 1}
9,"{'Minnesota': 1, 'Minneapolis': 1, 'America': 1}",{},{}


In [30]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [31]:
ct_ner['NER-msg']

2                                       []
3                                  [Texas]
5                                  [Texas]
7                            [Westchester]
9        [Minneapolis, America, Minnesota]
                       ...                
11741                                   []
11743                                   []
11744                                   []
11745                [Minneapolis, Albany]
11746                          [Wisconsin]
Name: NER-msg, Length: 7003, dtype: object

In [32]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
ct_ner['NER']

2                 [Southfield, Southfield]
3                           [Texas, Texas]
5                           [Texas, Texas]
7               [Westchester, Westchester]
9        [Minneapolis, America, Minnesota]
                       ...                
11741                           [New York]
11743                        [SAN ANTONIO]
11744                           [New York]
11745      [Minneapolis, Albany, New York]
11746               [Wisconsin, Wisconsin]
Name: NER, Length: 7003, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ner_lst = ct_ner['NER'].values

In [37]:
ner_lst

array([list(['southfield']), list(['texas']), list(['texas']), ...,
       list(['new york']), list(['minneapolis', 'new york', 'albany']),
       list(['wisconsin'])], dtype=object)

In [38]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [39]:
list

['southfield',
 'texas',
 'texas',
 'westchester',
 'minneapolis',
 'america',
 'minnesota',
 'minneapolis',
 'america',
 'minnesota',
 'portland',
 'los angeles',
 'minneapolis',
 'ahmaud arbery',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minnesota',
 'u.s.',
 'franklin township',
 'washington',
 'u.s.',
 'minneapolis',
 'mo',
 'tx',
 'oakland',
 'oakland',
 'vallejo',
 'minneapolis',
 'america',
 'minnesota',
 'the kingdom of god',
 'texas',
 'mn',
 'florid',
 'tennessee',
 'south carolina',
 'indiana',
 'washington',
 'd.c.',
 'utah',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'u.s.',
 'the united states',
 'arlington',
 'houston',
 'denver',
 'houston',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'minnesota',
 'minneapolis',
 'u.s.',
 'houston',
 'minneapolis',
 'syria',
 'us',
 'minneapolis',
 'houston',
 'paris',
 'd.c.',
 'madison',
 'stafford st.',
 'waupaca',
 '3:00pm',
 'whitewater',
 'milwaukee',
 'elkhorn',
 'wi',
 'genoa city',
 'plymouth',
 

In [40]:
c = collections.Counter()

In [41]:
dict_a=collections.Counter(list).most_common()

In [42]:
dict_a

[('minneapolis', 2128),
 ('houston', 703),
 ('america', 582),
 ('us', 464),
 ('u.s.', 442),
 ('washington', 357),
 ('the united states', 344),
 ('texas', 322),
 ('new york', 314),
 ('minnesota', 276),
 ('new jersey', 258),
 ('nj', 192),
 ('california', 159),
 ('chicago', 142),
 ('florida', 137),
 ('south jersey', 121),
 ('buffalo', 114),
 ('los angeles', 99),
 ('new york city', 93),
 ('wisconsin', 80),
 ('vatican city', 77),
 ('seattle', 74),
 ('atlanta', 73),
 ('d.c.', 70),
 ('oakland', 69),
 ('milwaukee', 66),
 ('georgia', 64),
 ('australia', 64),
 ('san francisco', 62),
 ('philadelphia', 62),
 ('north carolina', 59),
 ('boston', 55),
 ('franklinville', 53),
 ('virginia', 50),
 ('n.j.', 47),
 ('denver', 45),
 ('maryland', 45),
 ('st. louis', 41),
 ('london', 40),
 ('nyc', 40),
 ('richmond', 40),
 ('portland', 38),
 ('franklin township', 38),
 ('pearland', 37),
 ('russia', 37),
 ('alabama', 37),
 ('mn', 36),
 ('las vegas', 36),
 ('zoom', 36),
 ('columbus', 36),
 ('albany', 35),
 ('md.

In [43]:
dataframe610=pd.DataFrame(dict_a)

In [44]:
len(dict_a)

1531

In [45]:
dataframe610.to_csv('dataframe0610.csv')